# **ML - Pipeline de Dados**
**Autor:** [Anderson França](https://www.linkedin.com/in/anderson-m-franca/) | **Contato:** [github.com/andfranca](https://github.com/andfranca/estatistica-e-aprendizado-de-maquinas-ptbr)

<a href="https://creativecommons.org/licenses/by/4.0/deed.en"><img align="left" width="80" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc.png"/></a>

Tudo o que vimos até aqui continua fundamental para sucesso em nossas iniciativas, como:

- Avaliação dos modelos
- Métricas de Regressão e Classificação
- Validação Cruzada
- Ajustes de Hiperparâmetros

> Agora precisamos organizar tudo isso num fluxo profissional com pipelines de modelagem.

### **O que é um pipeline em ciência de dados?**

- Um pipeline é uma sequência organizada de etapas que processa dados de forma sistemática e automatizada.
- Em cada etapa, os dados são transformados, limpos, ou processados para atingir um objetivo específico, como análise, visualização ou treinamento de um modelo.


Na imagem abaixo, podemos observar um exemplo de pipeline em aprendizado de máquinas:



<img src= "https://www.oreilly.com/api/v2/epubs/9781783980284/files/assets/23512725-9f82-4ba2-ba00-1baafcae62e4.png" width = '800'/>

Imagem: [Understanding the machine learning workflow](https://www.oreilly.com/library/view/machine-learning-for/9781783980284/d2fe10b6-b66b-4f3d-b1c2-a85e4b617abc.xhtml)

Podemos citar como principais motivos para a utilização de um fluxo de desenvolvimento:
- Padronização do Processo
- Automação do processo
- Redução do tempo de desenvolvimento
- Flexibilidade
- Escalabilidade

### **Ferramentas para criar pipelines**

Para criar um pipeline, não é necessário utilizar nenhuma ferramenta específica, mas existem muitas ferramentas e bibliotecas que podem facilitar o processo. As bibliotecas e ferramentas mais populares são:


- **Scikit-learn:** Biblioteca de código aberto para Python, que oferece muitas ferramentas para pré-processamento de dados, seleção de recursos, treinamento e avaliação de modelos. O Scikit-learn é modular e pode ser facilmente integrado com outras ferramentas.

- **Apache Spark:** É um mecanismo de análise unificado para processamento de dados em grande escala com módulos integrados para SQL, streaming, machine learning e processamento de gráficos (fonte: [Cloud google](https://cloud.google.com/learn/what-is-apache-spark?hl=pt-br) ). O Spark oferece muitas ferramentas para processamento de dados em larga escala e pode ser facilmente integrado com outras ferramentas e bibliotecas.

- **TensorFlow:** É uma plataforma de machine learning de código aberto desenvolvida pelo Google. O TensorFlow é altamente escalável e pode ser usado para desenvolver pipelines de machine learning em grandes conjuntos de dados distribuídos.

- **PyTorch:** É uma biblioteca de machine learning de código aberto para Python, desenvolvida pelo Facebook. O PyTorch é altamente flexível e pode ser usado para desenvolver pipelines de machine learning em diferentes dispositivos, incluindo CPU, GPU e TPU.


Além disso, existem plataformas próprias para a criação de pipelines para aprendizado de máquinas, como:
-  KNIME
- H2O.ai
- RapidMiner
- Databricks

### **Pipeline com Scickit-learn**

Os pipelines no Scikit-learn são uma maneira poderosa de organizar fluxos de trabalho. Eles ajudam a manter o código limpo, padronizado e facilitam o ajuste de modelos. 

Podemos combinar transformadores nativos e personalizados para atender às necessidades específicas dos projetos.


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

#### **Vamos construir um pipeline**

A base de atrito representa dados de funcionários de uma organização. Nosso objetivo é prever quais funcionários têm maior chance de deixar o setor, com base em informações como idade, tempo de empresa, área de atuação, satisfação no trabalho, entre outras.


##### **Carregando a base de dados**
Antes de treinar o modelo, precisamos garantir que os dados estejam prontos para uso. Isso significa: limpar colunas irrelevantes, padronizar nomes e separar o que queremos prever (_target_) do que vamos usar como entrada (_features_).


**Etapas dessa preparação:**
1. **Ler o arquivo CSV** da base de atrito
2. **Remover colunas** que não contribuem para a previsão
    - **Exemplo:** Unnamed: 0 e ID (identificadores)
3. **Corrigir nomes de colunas** que tenham espaços ou variações
4. **Separar os dados** em:
    - `X`: informações dos funcionários (entradas)
    - `y`: Deixou a empresa (alvo)


In [ ]:
# Carregar a base
df = pd.read_csv("base_atrito.csv")

# Ajustar nomes de colunas e remover colunas irrelevantes
df = df.drop(columns=["Unnamed: 0", "ID"])

##### **O que temos na base:**

- Funcionários: dados de perfil, satisfação, desempenho e condições de trabalho

- **Colunas numéricas**:

    - `Idade`, `Anos na Empresa`, `Salário`, `Número de Dependentes`, etc.

- **Colunas categóricas**:

    -   `Gênero`, `Área de Atuação`, `Equilíbrio vida-trabalho`, `Trabalho remoto`, `Reputação da Empresa`, etc.

- **Variável alvo (target)**:

    - `Deixou a empresa` → 0 = Não saiu | 1 = Saiu




In [6]:
# Definir variável alvo
y = df["Deixou a empresa"]
X = df.drop(columns=["Deixou a empresa"])

Antes de construirmos o modelo, precisamos garantir que ele seja capaz de prever se um funcionário vai sair da empresa com base apenas em **dados que ele nunca viu antes**.

Por isso: 
- Vamos usar parte dos funcionários para o modelo aprender quem saiu e quem ficou.
- E reservar outra parte para testar se ele consegue prever corretamente o risco de saída em perfis novos.

In [8]:
# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### **Transformadores**

Os transformadores realizam tarefas de pré-processamento, como limpeza, normalização e extração de features. Podemos ter vários transformadores no pipeline, que serão executados em sequência.


**Exemplos**
- Preenchimento de valores ausentes: `SimpleImputer`.
- Escalonamento: `StandardScaler`, `MinMaxScaler`.
- Codificação de dados categóricos: `OneHotEncoder`, `OrdinalEncoder`.
- Seleção de variáveis: `SelectKBest`, `VarianceThreshold`.
- Redução de dimensionalidade: `PCA`, `TruncatedSVD`.

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

##### **Tratando variáveis numéricas e categóricas**

Os algoritmos de machine learning não trabalham diretamente com textos, e podem ter dificuldade com variáveis em escalas muito diferentes.

Por isso, precisamos preparar os dados de forma adequada antes de treinar o modelo.

O que temos na base:
- **Variáveis numéricas:** `Idade`, `Salário`, `Anos na Empresa`, etc.
- **Variáveis categóricas:** `Gênero`, `Área de Atuação`, `Equilíbrio Vida-Trabalho`, etc.

Para simplificar, vamos transformar as variáveis da seguinte maneira: 
- **Numéricas** → `StandardScaler`  (padroniza os valores para média 0 e desvio 1)
- **Categóricas** → `OneHotEncoder` (transforma texto em colunas binárias: 0 ou 1)


In [10]:
# Identificar colunas numéricas e categóricas
colunas_numericas = X.select_dtypes(include=np.number).columns.tolist()
colunas_categoricas = X.select_dtypes(include="object").columns.tolist()

Para realizar toda a transformação, vamos utilizar o `ColumnTransformer`

**Resultado:** Cada variável será tratada da forma certa, e o modelo receberá os dados já prontos para aprendizado.

In [14]:
#### Criar o pré-processamento
preprocessador = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), colunas_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), colunas_categoricas)
    ]
)

##### **Criando o Pipeline**

Agora que já configuramos o pré-processamento (escalonamento para variáveis numéricas e codificação para as categóricas), vamos integrar tudo isso em um pipeline de machine learning. Note que adicionamos uma etapa chamada modelo, que é o nosso **estimador final** que será o responsável por fazer as previsões

O que vamos incluir no pipeline:
- Pré-processamento dos dados com `ColumnTransformer`
- Modelo de classificação — aqui usaremos `RandomForestClassifier`
- Tudo encapsulado em um único objeto, simples de usar e reaproveitar


In [15]:
# Criar pipeline
pipeline = Pipeline(steps=[
    ("preprocessamento", preprocessador),
    ("modelo", RandomForestClassifier(random_state=42))
])

Após treinar o pipeline, usamos o modelo para fazer previsões nos dados de teste e avaliamos o desempenho com:

In [24]:
# Treinar pipeline
pipeline.fit(X_train, y_train)

,steps,"[('preprocessamento', ...), ('modelo', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [25]:
y_pred = pipeline.predict(X_test)
relatorio = classification_report(y_test, y_pred, output_dict=False)
print(relatorio)

              precision    recall  f1-score   support

           0       0.76      0.77      0.76      7804
           1       0.74      0.73      0.74      7096

    accuracy                           0.75     14900
   macro avg       0.75      0.75      0.75     14900
weighted avg       0.75      0.75      0.75     14900



### **Salvando o modelo**

Salvar modelos é uma etapa essencial no ciclo de desenvolvimento de aprendizado de máquina, com benefícios significativos tanto em termos de eficiência quanto de escalabilidade. 

Geralmente, salvamos os modelos quando queremos:
- Reutilizar modelos treinados sem re-treinar
- Economizar tempo em aplicações de produção
- Manter consistência em experimentos e previsões


##### **Joblib**

**Joblib** é uma biblioteca projetada para otimizar o processo de salvar e carregar objetos grandes e complexos, especialmente em contextos de aprendizado de máquina e ciência de dados. 

Ela se destaca pela eficiência ao lidar com dados volumosos, como arrays NumPy e matrizes esparsas, que são comuns em modelos e pipelines do Scikit-learn.


In [21]:
import joblib

O método `joblib.dump` é usado para salvar objetos Python em arquivos, permitindo que sejam carregados posteriormente para reutilização. Ele é muito usado em ML para serializar (salvar) modelos treinados, pipelines ou outros objetos grandes, como arrays `NumPy`.

In [22]:
# Salvar o pipeline já ajustado
joblib.dump(pipeline, "modelo_pipeline.pkl")

['modelo_pipeline.pkl']

##### **Como ele funciona?**

**Entrada:**
O método recebe o objeto a ser salvo (por exemplo, um modelo treinado ou pipeline) e o caminho do arquivo onde ele será armazenado.

**Saída:**
Ele salva o objeto no formato binário no local especificado, geralmente com a extensão .pkl ou .joblib.

**Uso:**
Ao salvar um modelo ou pipeline, todo o estado interno do objeto, incluindo os pesos, hiperparâmetros e configurações, é armazenado no arquivo.


In [24]:
# Carregar o modelo salvo posteriormente
modelo_carregado = joblib.load("modelo_pipeline.pkl")

In [ ]:
# Usando com dados novos
dados_novos = pd.read_csv("novos_dados.csv")

# Fazer previsões com o modelo carregado
previsoes = modelo_carregado.predict(dados_novos)

In [ ]:
# Adicionar previsões ao DataFrame de dados novos
dados_novos["Previsao"] = previsoes

In [ ]:
# Visualizar as previsões
dados_novos

,Idade,Genero,Anos na Empresa,Area de Atuacao,Salario,Equilibrio vidatrabalho,Satisfacao Trabalho,Taxa de desempenho,Numero de promocoes,Hora Extra,...,Numero de Dependentes,Nivel no Trabalho,Tamanho da empresa,Tempo de empresa meses,Trabalho remoto,Oportunidade de Lideranca,Inovacao,Reputacao da Empresa,Reconhecimento Funcionarios,Previsao
0,37,Feminino,27,Tecnologia,12617,Razoavel,Alta,Media,1,Sim,...,2,Junior,Pequena,57,Sim,Nao,Nao,Excelente,Alta,1
1,35,Masculino,12,Educacao,5935,Ruim,Alta,Media,2,Nao,...,1,Senior,Media,19,Nao,Nao,Nao,Ruim,Alta,1
2,52,Masculino,34,Educacao,3908,Ruim,Muito Alta,Media,1,Nao,...,2,Pleno,Media,63,Nao,Sim,Nao,Ruim,Media,0
3,35,Masculino,21,Saude,5663,Boa,Media,Media,0,Sim,...,2,Senior,Media,70,Nao,Nao,Nao,Razoavel,Media,0
4,30,Masculino,4,Saude,8184,Razoavel,Alta,Alta,4,Nao,...,3,Pleno,Pequena,50,Nao,Nao,Nao,Ruim,Baixa,0
5,25,Masculino,14,Saude,9156,Boa,Alta,Media,1,Nao,...,4,Junior,Pequena,18,Sim,Nao,Nao,Boa,Media,0
6,49,Feminino,4,Tecnologia,9692,Excelente,Alta,Media,0,Nao,...,2,Pleno,Media,59,Nao,Nao,Nao,Ruim,Baixa,0
7,23,Feminino,4,Tecnologia,10004,Excelente,Alta,Abaixo da Media,1,Nao,...,0,Pleno,Grande,44,Nao,Nao,Nao,Ruim,Baixa,1
8,38,Feminino,14,Educacao,4493,Boa,Alta,Alta,2,Nao,...,0,Junior,Pequena,38,Nao,Nao,Sim,Boa,Media,1
9,32,Masculino,22,Midia,6011,Boa,Muito Alta,Alta,0,Nao,...,0,Junior,Pequena,88,Nao,Nao,Nao,Razoavel,Baixa,1


### **Referências**

Scikit-learn – Model Selection. https://scikit-learn.org/stable/model_selection.html 

Scikit-learn – ColumnTransformer
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

Scikit-learn – Pipeline
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

Scikit-learn – make_pipeline
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html

Scikit-learn – SimpleImputer
https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

Scikit-learn – OneHotEncoder
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Scikit-learn – OrdinalEncoder
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html

Scikit-learn – StandardScaler
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

Scikit-learn – RandomForestClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Scikit-learn – cross_val_score
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

Scikit-learn – Cross-validation: evaluating estimator performance
https://scikit-learn.org/stable/modules/cross_validation.html

Scikit-learn – GridSearchCV
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Scikit-learn – RandomizedSearchCV
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

Scikit-learn – Joblib (persisting models)
https://scikit-learn.org/stable/model_persistence.html

Scikit-learn – Cross-validation: evaluating estimator performance.  
https://scikit-learn.org/stable/modules/cross_validation.html 

#### Apêndice

In [ ]:
# Adicionando mais etapas de pré-processamento

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Separar colunas
colunas_numericas = X.select_dtypes(include="number").columns.tolist()
colunas_categoricas = X.select_dtypes(include="object").columns.tolist()

# Pipeline para numéricos: imputação + escalonamento
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

# Pipeline para categóricos: imputação + encoding
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="-1")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# ColumnTransformer com ambos
preprocessador = ColumnTransformer(transformers=[
    ("num", pipeline_num, colunas_numericas),
    ("cat", pipeline_cat, colunas_categoricas)
])

In [19]:
# Criar pipeline
pipeline = Pipeline(steps=[
    ("preprocessamento", preprocessador),
    ("modelo", RandomForestClassifier(random_state=42))
])

In [20]:
# Treinar pipeline
pipeline.fit(X_train, y_train)

,steps,"[('preprocessamento', ...), ('modelo', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
